In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
c = np.load('/content/drive/MyDrive/Colab Notebooks/dataset.npy')
c.shape

(26953, 2048)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/output.csv')

data = df['verdict'].tolist()

In [ ]:
import numpy as np
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/dataset.npy')
y_train = data[14::]
X_test = np.load('/content/drive/MyDrive/Colab Notebooks/public_test.npy')

In [ ]:
from sklearn.preprocessing import MinMaxScaler #fixed import

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
train_labels_encoded = le.transform(y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

clf = RandomForestClassifier(n_estimators=1000, max_depth=500, random_state=42)

for i in tqdm(range(1000), desc="Training Progress"):
    clf.set_params(n_estimators=i+1)
    clf.fit(X_train, train_labels_encoded)



Training Progress:   8%|▊         | 85/1000 [1:38:05<33:52:10, 133.26s/it]

In [ ]:
import joblib
joblib.dump(clf, '/content/drive/MyDrive/Colab Notebooks/model.pkl')

In [ ]:
import joblib
model = joblib.load(open(r'/content/drive/MyDrive/Colab Notebooks/model.pkl','rb'))

In [ ]:
import json
with open("/content/drive/MyDrive/Colab Notebooks/ise-dsc01-public-test-offcial.json",encoding="utf-8") as file:
  dataset = json.load(file)

In [ ]:
testset = [arr.reshape(1, -1) for arr in X_test]
testset = np.array(testset)

In [ ]:
import re


result_dict = {}

# Loop through each item in the dataset
i = 0
for key, data in dataset.items():
    # Extract context and claim from the current item
    context = data['context']
    claim = data['claim']
    inp = testset[i]
    pred = model.predict(inp)

    if pred[0] == 2:
        output = "SUPPORTED"
    elif pred[0] == 1:
        output = "REFUTED"
    else:
        output = "NEI"

    i = i+1
    # Store the results in the result_dict
    result_dict[key] = {
        "context": context,
        "claim": claim,
        "verdict": output
    }

with open('resultsbkai.json', 'w', encoding='utf-8') as output_file:
    json.dump(result_dict, output_file, ensure_ascii=False, indent=2)

In [ ]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
import numpy as np
from typing import List
from transformers import logging


logging.set_verbosity(logging.ERROR)
PRE_TRAIN_MODEL = ("vinai/phobert-base", "vinai/phobert-large", "vinai/phobert-base-v2")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import TFAutoModel, AutoTokenizer

PhoBERTtokenizer = AutoTokenizer.from_pretrained(PRE_TRAIN_MODEL[1])
PhoBERTtokenizer.model_max_length = 256
PhoBERTmodel = TFAutoModel.from_pretrained(PRE_TRAIN_MODEL[1])

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded =  tf.cast(tf.broadcast_to(tf.expand_dims(attention_mask, -1), token_embeddings.shape), tf.float32)
    return tf.reduce_sum(token_embeddings * input_mask_expanded, 1) / tf.clip_by_value(tf.reduce_sum(input_mask_expanded, 1), clip_value_min=1e-9, clip_value_max=np.inf)

def get_sentence_embedding(model, tokenizer, sentences: str | List[str] | np.ndarray):
    if isinstance(sentences, str):
        sentences = [sentences,]

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='tf')

    # Compute token embeddings
    model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = tf.keras.utils.normalize(sentence_embeddings)

    return sentence_embeddings.numpy()

In [ ]:
import json
with open("resultsbkai.json",encoding="utf-8") as file:
  dataset = json.load(file)

In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d8f845c63ef4feb247b8a18eee92b5f63e4d376bedf940827c5a97f6437f8f46
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
import nltk
from sentence_transformers import SentenceTransformer, util
import json

# Initialize the NLTK sentence tokenizer
nltk.download('punkt')
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

result_dict = {}
i = 0

model = SentenceTransformer("sentence-transformers/paraphrase-xlm-r-multilingual-v1")

# Loop through each item in the dataset
for key, data in dataset.items():
    # Extract context and claim from the current item
    context = data['context']
    claim = data['claim']
    verdict = data['verdict']

    # Split context into sentences using NLTK
    sentences = sentence_tokenizer.tokenize(context)

    # Compute embeddings for claim
    claim_embedding = model.encode(claim, convert_to_tensor=True)

    # Compute embeddings for context sentences
    context_embeddings = model.encode(sentences, convert_to_tensor=True)

    if verdict != "NEI":
        # Compute cosine-similarities between claim and each context sentence
        cosine_scores = util.pytorch_cos_sim(claim_embedding, context_embeddings)

        # Find the sentence with the highest cosine similarity score
        max_score_index = int(cosine_scores.argmax())
        max_score_sentence = sentences[max_score_index]
    else:
        max_score_sentence = ""

    i = i + 1

    # Store the results in the result_dict
    result_dict[key] = {
        "verdict": verdict,
        "evidence": max_score_sentence  # Set evidence to the highest-scoring sentence
    }

with open('final.json', 'w', encoding='utf-8') as output_file:
    json.dump(result_dict, output_file, ensure_ascii=False, indent=2)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
pred = model.predict(X_test)
y_pred = le.inverse_transform(pred)

y_pred

array(['SUPPORTED', 'NEI', 'NEI', ..., 'REFUTED', 'REFUTED', 'REFUTED'],
      dtype='<U9')

In [ ]:
model =

In [ ]:
# Split context into sentences
sentences = re.split(r'\. |\n\n|(?<=\.”)', context)

# Compute embeddings for claim
claim_embedding = model.encode("Hà Nội và Sài Gòn xa nhau", convert_to_tensor=True)

# Compute embeddings for context sentences
context_embeddings = model.encode("sentences", convert_to_tensor=True)

cosine_scores = util.pytorch_cos_sim(claim_embedding, context_embeddings)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout,Embedding

embedding_vector_features = 2048

model=Sequential()

model.add(LSTM(128,input_shape=(c.shape),activation='relu'))

model.add(Dropout(0.2))

model.add(LSTM(128,activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(32,activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(4,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 128)